<a href="https://colab.research.google.com/github/barrioBDes/creditCardFraud/blob/main/creditCardUV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Comprensión de negocio


La seguridad a la hora de realizar pagos con tarjeta de crédito es un asunto de gran interés tanto para entidades como para ladrones. Una entidad segura ahorrará dinero en indemnizaciones y ofrecerá un incentivo a quienes quieran tener seguros sus ahorros. Sobre este tema, hemos descargado un dataset de Kaggle conformado por 28 variables ofuscadas por razones de privacidad (o eso dicen), mediante Análisis de Componentes Principales.

Hay dos variables que sí están disponibles, y son Time y Amount.

De la variable Time no conocemos el punto de referencia (la transferencia 1), pero sí los minutos que han pasado desde ella. Para aprovechar el dato y convertirlo en algo útil para el modelo dividimos las operaciones en grupos de 3 horas, de forma que los horarios 'de noche' siempre sean de noche, y los 'de día', también estén marcados.

La variable Amount también está marcada, entre operaciones de 'hasta 60 euros', entre 60 y 2000 y más de 2000. De este modo tenemos en cuenta operaciones contactless y operaciones muy altas (al menos, para mi criterio).

### Librerías

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from google.colab import drive
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import statsmodels.formula.api as smf
import statsmodels.api as sm
import lightgbm as lgb
import seaborn as sn
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
from sklearn.metrics import confusion_matrix
import keras
from keras.layers import Input,Conv2D,MaxPooling2D,UpSampling2D, Dense
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers
from keras.models import Model
from keras.optimizers import RMSprop
from matplotlib import pyplot
!pip install sklearn-contrib-py-earth
from pyearth import Earth

drive.mount('/content/drive')

In [ ]:
def matrizConfusion(cm):
  print(cm)
  sn.heatmap(cm/np.sum(cm), annot=True, fmt='.2%', cmap='Blues')

def AUC(fpr, tpr, threshold):
  roc_auc = metrics.auc(fpr, tpr)
  plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
  plt.legend(loc = 'lower right')
  plt.plot([0, 1], [0, 1],'r--')
  plt.xlim([0, 1])
  plt.ylim([0, 1])
  plt.show()

def comprobarNAs(data):
  v = 0
  for _ in data.columns:
    if (data[_].isnull().sum() != 0):
      print(_ + ' contiene NAs o NULL')
      v = 1
  if (v == 0):
    print("No hay NAs en todo el dataset")
  return data

def nuevaVariable1(data):
  condiciones1 = [
       (data['Time']/3600/2 < 3),
       (data['Time']/3600/2 < 6),
       (data['Time']/3600/2 < 9),
       (data['Time']/3600/2 < 12),
       (data['Time']/3600/2 < 15),
       (data['Time']/3600/2 < 18),
       (data['Time']/3600/2 < 21),
       (data['Time']/3600/2 < 24)]
  valor1 = ['A','B','C','D','E','F','G','H']
  data['Hora'] = np.select(condiciones1, valor1, default='K')
  data = pd.get_dummies(data, prefix=['Hora'])
  data = data.drop(['Time','Hora_H'], axis=1)
  return data

def nuevaVariable2(data):
  condiciones2 = [
       (data['Amount'] < 60),
       ((data['Amount'] >= 60) & (data['Amount'] <=2000)),
       (data['Amount'] >2000)]
  valor2 = ['A','B','C']
  data['Cantidad'] = np.select(condiciones2, valor2, default='K')
  data = pd.get_dummies(data, prefix=['Cantidad'])
  data = data.drop(['Amount','Cantidad_C'], axis=1)
  clase = data['Class']
  data.drop(labels=['Class'], axis=1,inplace = True)
  data.insert(len(list(data)), 'Class', clase)
  return data

def resampleyMinMax(data):
  dataFraude = data.loc[data['Class'] == 1]
  dataNoFraude = data.loc[data['Class'] == 0].sample(len(dataFraude)*4)
  data = pd.concat([dataFraude, dataNoFraude])
  scaler = MinMaxScaler(feature_range=(-1, 1))
  dataEscalada = scaler.fit_transform(data)
  nombresColumna = data.columns
  data = pd.DataFrame(dataEscalada)
  data.columns = nombresColumna
  data = data.sample(frac=1).reset_index(drop=True)
  X = np.array(data.loc[:, data.columns != 'Class'])
  y = np.array(data.loc[:, data.columns == 'Class'])
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
  X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.10, random_state=1) 
  sm = SMOTE(random_state=2)
  X_train_res, y_train_res = sm.fit_sample(X_train, y_train.ravel())
  print('Tras el resamplig, hay ' + str(sum(y_train_res==1)) + ' positivos y ' + str(sum(y_train_res==-1)) + ' negativos.')
  return X_train, X_test, y_train, y_test, X_train, X_val, y_train, y_val, X_train_res, y_train_res

In [ ]:
data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/creditcard.csv")

## Comprensión de los datos

Los datos, como hemos comentado, se componen de 28 columnas ofuscadas, una variable llamada Time y otra llamada Amount. Transformamos las dos últimas y comprobamos que no existen NAs en el resto. En las siguientes líneas hacemos lo que hemos comentado, y demás escalamos y normalizamos las variables (por si no lo estuvieran ya con el PCA)

In [ ]:
data.head()

In [ ]:
data = comprobarNAs(data)
data = nuevaVariable1(data)
data = nuevaVariable2(data)
X_train, X_test, y_train, y_test, X_train, X_val, y_train, y_val, X_train_res, y_train_res = resampleyMinMax(data)

## Modelos (1 - LightGBM)

Cuando hacemos una predicción de clasificación con un modelo como LightGBM (o autoencoder, como veremos después), a nivel interno el algoritmo no decide "1" o "0", sino un scoring entre ambos que indica "cómo de probable es que sea "1" o "0". Llevamos a cabo 100 rondas de LightGBM y tomamos únicamente la que mejor AUC parece devolver.

In [ ]:
train_data = lgb.Dataset(X_train_res, label=y_train_res)
test_data = lgb.Dataset(X_test, label=np.reshape(y_test, (1, len(y_test)))[0])

parameters = {
    'application': 'binary',
    'objective': 'binary',
    'metric': 'auc',
    'is_unbalance': 'false',
    'boosting': 'gbdt',
    'num_leaves': 31,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.5,
    'bagging_freq': 20,
    'learning_rate': 0.05,
    'verbose': 0
}

lightgbm = lgb.train(parameters,
                       train_data,
                       valid_sets=test_data,
                       num_boost_round=100,
                       early_stopping_rounds=10)

## Evaluación LightGBM

Como la categoría de clasificación es en realidad un número entre 0 y 1, dependiendo de dónde establezcamos el corte tendremos una precisión u otra. Esto es lo que evalúa la curva ROC, dibujando el porcentaje de aciertos que tendríamos para cada nivel de corte; calculando su área tenemos un indicador de precisión del modelo. Hemos dividido el dataset en Train, Test y Validación para todos estos cálculos, las evaluaciones se hacen siempre sobre Validación.

In [ ]:
ypred = lightgbm.predict(X_val, num_iteration=lightgbm.best_iteration)
yval = np.reshape(y_val, (1, len(y_val)))
fpr, tpr, threshold = metrics.roc_curve(np.resize(y_val, (1,len(y_val)))[0], ypred)
AUC(fpr, tpr, threshold)

En la siguiente matriz de confusión observamos que devuelve 3 predicciones negativas que realmente fueran positivas (caso más desfavorable) y 0 'falsas alarmas' en las que predigamos un positivo que finalmente no lo es.

In [ ]:
valoresRealesValidacion = np.where(np.resize(y_val, (1,len(y_val)))[0]>0,1,0)
valoresPredichosValidac = np.where(ypred>0.55,1,0)
cm = confusion_matrix(valoresRealesValidacion,valoresPredichosValidac)
matrizConfusion(cm)

## Modelo 2 - Red Neuronal Autoencoder

Después de varias pruebas de configuración de capas y neuronas, el modelo parece converger en un 20% de precisión en 'test' en un modelo con claro sobreajuste (¿pocos datos?). 

Todo esto a juzgar por los gráficos devueltos por el History del modelo. No obstante podría tratarse de un error de escalas con la variable a predecir en Test, ya que finalmente su precisión es buena según la matriz de confusión y la curva ROC.

In [ ]:
nb_epoch = 60 #60
batch_size = 32 #16
learning_rate = 0.05 #0.01
input_dim = X_train.shape[1]

input_layer = Input(shape=(input_dim, ))
encoder = Dense(30, activation="relu", activity_regularizer=regularizers.l1(learning_rate))(input_layer)
decoder = Dense(8, activation="relu")(encoder)
decoder = Dense(2, activation="sigmoid")(encoder)
decoder = Dense(8, activation="relu")(decoder)
decoder = Dense(30, activation="relu")(decoder)
decoder = Dense(1, activation="sigmoid")(decoder)
autoencoder = Model(inputs=input_layer, outputs=decoder)
autoencoder.summary()

In [ ]:
y_trainCero = np.where(np.resize(y_train, (1,len(y_train)))[0]>0,1,0)
autoencoder.compile(metrics=['accuracy'],
                    loss='mean_squared_error',
                    optimizer='adam')

cp = ModelCheckpoint(filepath="autoencoder_classifier.h5",
                               save_best_only=True,
                               verbose=0)
                               
modelo = autoencoder.fit(X_train, y_trainCero,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(X_test, y_test),
                    verbose=1,
                    callbacks=[cp])
                  

In [ ]:
print(modelo.history.keys())

plt.plot(modelo.history['accuracy'])
plt.plot(modelo.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(modelo.history['loss'])
plt.plot(modelo.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

## Evaluación RN


Se dan 9 casos en los que hay fraude pero indica que no, y 0 en los que avise de fraude y finalmente no lo haya. La el área bajo la curva ROC ronda el 0.96

In [ ]:
valoresRealesValidacion = np.where(np.resize(y_val, (1,len(y_val)))[0]>0,1,0)
valoresPredichosValidac = np.where(np.resize(modelo.model.predict(X_val), (1,len(modelo.model.predict(X_val))))>0.8,1,0)[0]
cm = confusion_matrix(valoresRealesValidacion,valoresPredichosValidac)
matrizConfusion(cm)

In [ ]:
fpr, tpr, threshold = metrics.roc_curve(valoresRealesValidacion, valoresPredichosValidac)
AUC(fpr, tpr, threshold)

## Stacking - modelo Earth

Con las predicciones devueltas por el LightGBM y la red neuronal (la predicción real, no su traducción a 0 - 1, llevamos a cabo un Stacking con un modelo Earth.

Para ello creamos un dataset con 197 registros, en el que añadimos el GBM predicho, el Autoencoder predicho, y el cuadrado y cubo de ambos, para que la nueva predicción pueda realizar splines. Además incluimos la variable real (1 o 0). Normalizamos y estandarizamos todo.

In [ ]:
prediccionGBMval = lightgbm.predict(X_val, num_iteration=lightgbm.best_iteration)
prediccionENCval = np.resize(modelo.model.predict(X_val), (1,len(modelo.model.predict(X_val))))[0]
yValidacion = np.where(np.resize(y_val, (1,len(y_val)))[0]>0,1,0)
prediccionesVal = pd.DataFrame({"GBMpredicho": prediccionGBMval, "ENCpredicho": prediccionENCval, "validacionReal": yValidacion})
prediccionesVal['GBMpredichoCuadrado'] = prediccionesVal['GBMpredicho']*prediccionesVal['GBMpredicho']
prediccionesVal['GBMpredichoCubo'] = prediccionesVal['GBMpredicho']*prediccionesVal['GBMpredicho']*prediccionesVal['GBMpredicho']
prediccionesVal['ENCpredichoCuadrado'] = prediccionesVal['ENCpredicho']*prediccionesVal['ENCpredicho']
prediccionesVal['ENCpredichoCubo'] = prediccionesVal['ENCpredicho']*prediccionesVal['ENCpredicho']*prediccionesVal['ENCpredicho']
prediccionesVal = prediccionesVal.reindex(columns=['GBMpredicho','GBMpredichoCuadrado','GBMpredichoCubo','ENCpredicho','ENCpredichoCuadrado','ENCpredichoCubo','validacionReal'])
scaler = MinMaxScaler(feature_range=(0, 1))
prediccionesValEscalados = scaler.fit_transform(prediccionesVal)
nombresColumna = prediccionesVal.columns
prediccionesVal = pd.DataFrame(prediccionesValEscalados)
prediccionesVal.columns = nombresColumna
prediccionesVal

In [ ]:
X = np.array(prediccionesVal.loc[:, prediccionesVal.columns != 'validacionReal'])
y = np.array(prediccionesVal.loc[:, prediccionesVal.columns == 'validacionReal'])

model = Earth()
model.fit(X,y)
    
print(model.trace())
print(model.summary())

## Evaluación stacking (Earth)

La curva ROC tiene un área por debajo de 1.00 como resultado de haber combinado el modelo de LightGBM con la red neuronal.

In [ ]:
EarthPrediccion = model.predict(X)
scaler = MinMaxScaler(feature_range=(0, 1))
EarthPrediccionRango = scaler.fit_transform(pd.DataFrame({"EarthPrediccion": EarthPrediccion}))
EarthPrediccionRango = np.resize(EarthPrediccionRango, (1,len(EarthPrediccionRango)))
valoresRealesValidacion = np.where(np.resize(y_val, (1,len(y_val)))[0]>0,1,0)
valoresPredichosValidac = np.where(EarthPrediccionRango>0.75,1,0)[0]
cm = confusion_matrix(valoresRealesValidacion,valoresPredichosValidac)
matrizConfusion(cm)

In [ ]:
fpr, tpr, threshold = metrics.roc_curve(np.where(np.resize(y_val, (1,len(y_val)))[0]>0,1,0), EarthPrediccionRango[0])
AUC(fpr, tpr, threshold)

## Evaluación stacking - GLM

Tal y como hemos hecho con el Earth, lanzamos un GLM. Este modelo es más explicativo que el anterior, y gracias a los p-valores podemos saber que la precisión del autoencoder es más fiable que la del GBM y se tiene más en cuenta para calcular el scoring final.

In [ ]:
!pip install PyGLM
import statsmodels.api as sm
import glm
formula = 'validacionReal ~ GBMpredicho + GBMpredichoCuadrado + GBMpredichoCubo + ENCpredicho + ENCpredichoCuadrado + ENCpredichoCubo'
glm = smf.glm(formula = formula, data=prediccionesVal, family=sm.families.Binomial())
result = glm.fit()
print(result.summary())

In [ ]:
X = pd.DataFrame(X)
X.columns = nombresColumna[:-1]
GLM = result.predict(X)
fpr, tpr, threshold = metrics.roc_curve(np.where(np.resize(y_val, (1,len(y_val)))[0]>0,1,0), GLM)
AUC(fpr, tpr, threshold)

In [ ]:
cm = confusion_matrix(valoresRealesValidacion,np.where(np.resize(GLM.to_numpy(), (1,len(GLM.to_numpy())))[0]>0.1,1,0))
matrizConfusion(cm)

## GLM sin variables ruidosas

In [ ]:
formula = 'validacionReal ~ ENCpredicho + ENCpredichoCuadrado + ENCpredichoCubo'
glm = smf.glm(formula = formula, data=prediccionesVal, family=sm.families.Binomial())
result = glm.fit()
print(result.summary())

In [ ]:
X = pd.DataFrame(X)
X.columns = nombresColumna[:-1]
GLM = result.predict(X)
fpr, tpr, threshold = metrics.roc_curve(np.where(np.resize(y_val, (1,len(y_val)))[0]>0,1,0), GLM)
AUC(fpr, tpr, threshold)

In [ ]:
cm = confusion_matrix(valoresRealesValidacion,np.where(np.resize(GLM.to_numpy(), (1,len(GLM.to_numpy())))[0]>0.1,1,0))
matrizConfusion(cm)